In [ ]:
!git clone https://github.com/GATECH-EIC/TinyML-Contest-Solution.git

Cloning into 'TinyML-Contest-Solution'...
remote: Enumerating objects: 30547, done.
remote: Counting objects: 100% (2/2), done.
remote: Total 30547 (delta 0), reused 2 (delta 0), pack-reused 30545
Receiving objects: 100% (30547/30547), 153.75 MiB | 7.53 MiB/s, done.
Resolving deltas: 100% (84/84), done.
Updating files: 100% (30513/30513), done.


In [ ]:
%cd TinyML-Contest-Solution/Training

/content/TinyML-Contest-Solution/Training


In [ ]:
# -*- coding: utf-8 -*-
"""arritmia_cardiaca.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1kIePtG_yixpzcbjVl5mY7nwQkAytTe_V
"""

# Credits to https://towardsdatascience.com/multi-task-learning-for-computer-vision-classification-with-keras-36c52e6243d2

# Imports
print("start imports")
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import time
import os
import csv
import random
from tensorflow import keras
print("end imports")


# Custom stuff from https://github.com/GATECH-EIC/TinyML-Contest-Solution/blob/master/Training/select_model.py
#from helpers.cosine_annealing import CosineAnnealingScheduler
from swa.tfkeras import SWA

# explicit function to normalize array
def normalize(arr, t_min=0, t_max=1):
    norm_arr = []
    diff = t_max - t_min
    diff_arr = max(arr) - min(arr)
    for i in arr:
        temp = (((i - min(arr))*diff)/diff_arr) + t_min
        norm_arr.append(temp)
    return norm_arr

""" Functions for loading data """
def txt_to_numpy(filename, row=1250):
    file = open(filename)
    lines = file.readlines()
    datamat = np.arange(row, dtype=np.float)
    row_count = 0
    for line in lines:
        line = line.strip().split(' ')
        datamat[row_count] = line[0]
        row_count += 1

    return datamat

"""
AFb,Atrial Fibrillation
AFt,Atrial Flutter
SR,Sinus Rhythm
SVT,Supraventricular Tachycardia
VFb,Ventricular Fibrillation
VFt,Ventricular Flutter
VPD,Ventricular Premature Depolarizations
VT,Ventricular Tachycardia
"""
def txt_to_disease_type(txt):
    txt = txt.upper()
    if txt == "AFB":
      return [1, 0, 0, 0, 0, 0, 0, 0]
    if txt == "AFT":
      return [0, 1, 0, 0, 0, 0, 0, 0]
    if txt == "SR":
      return [0, 0, 1, 0, 0, 0, 0, 0]
    if txt == "SVT":
      return [0, 0, 0, 1, 0, 0, 0, 0]
    if txt == "VFB":
      return [0, 0, 0, 0, 1, 0, 0, 0]
    if txt == "VFT":
      return [0, 0, 0, 0, 0, 1, 0, 0]
    if txt == "VPD":
      return [0, 0, 0, 0, 0, 0, 1, 0]
    if txt == "VT":
      return [0, 0, 0, 0, 0, 0, 0, 1]
    raise "A string was not recognized as a valid input on txt_to_disease_type func!"

"""
def read_data(csv_path, imgs_folder="./tinyml_contest_data_training"):
    x, y1, y2 = [], [], []
    with open(csv_path, "r") as csv_file:
      reader = csv.reader(csv_file)
      next(reader)
      for item in reader: # item[0] = label; item[1] = filename
        x.append(txt_to_numpy(os.path.join(imgs_folder, item[1]), 1250))
        y1.append([1, 0] if int(item[0]) == 0 else [0, 1]) # Label for life threat
        y2.append(txt_to_disease_type(item[1].split("-")[1]))
    x, y1, y2 = sklearn.utils.shuffle(x, y1, y2)
    return np.array(x), np.array(y1), np.array(y2)
"""
def read_data(csv_path, imgs_folder="./tinyml_contest_data_training", augmentation=False, flip_peak=True, flip_time=True, add_noise=True): #, flip_peak=False, flip_time=False, add_noise=False):
    x, y1, y2, y3, y4, y5 = [], [], [], [], [], []
    print("passed here")
    with open(csv_path, "r") as csv_file:
      reader = csv.reader(csv_file)
      next(reader)
      print("Reading data...")
      counter = 0
      for item in reader: # item[0] = label; item[1] = filename
        x.append(txt_to_numpy(os.path.join(imgs_folder, item[1]), 1250))
        y1.append([1, 0] if int(item[0]) == 0 else [0, 1]) # Label for life threat
        y2.append(txt_to_disease_type(item[1].split("-")[1]))
        counter +=1
        if counter % 200 == 0:
          print(f"{counter} entries read...")
    x, y1, y2 = sklearn.utils.shuffle(x, y1, y2)
    x = np.array(x)

    # y3 creation
    for sample in x:
       y3.append(np.mean(sample))
       stdeviation = np.std(sample)
       y4.append(stdeviation)
       y5.append(stdeviation ** 2)

    if augmentation:
      #x = x + np.random.normal(0, random.random()*np.amax(x)*0.05, (len(x), 1))

      for i in range(len(x)):
        flip_p = random.random()
        flip_t = random.random()
        if flip_p < 0.5 and flip_peak:
          x[i] = -x[i]
        if flip_t < 0.5 and flip_time:
          x[i] = np.flip(x[i])
        if add_noise:
          print(f"i={i}")
          max_peak = x[i].max() * 0.05
          factor = random.random()
        # factor = 1
          noise = np.random.normal(0, factor * max_peak, x[0].shape)
          x[i] = x[i] + noise
    return x, np.array(y1), np.array(y2), np.array(y3), np.array(y4), np.array(y5)


""" Loading data """
x_train, y_train_1, y_train_2, y_train_3, y_train_4, y_train_5 = read_data("./data_indices/train_indice.csv", augmentation=False) #augmentation=True, flip_peak=True, flip_time=True, add_noise=True)
x_test, y_test_1, y_test_2, y_test_3, y_test_4, y_test_5 = read_data("./data_indices/test_indice.csv")
print("Finished reading data!")


start imports
end imports
passed here
Reading data...
200 entries read...


<ipython-input-3-40b512373d09>:44: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  datamat = np.arange(row, dtype=np.float)


400 entries read...
600 entries read...
800 entries read...
1000 entries read...
1200 entries read...
1400 entries read...
1600 entries read...
1800 entries read...
2000 entries read...
2200 entries read...
2400 entries read...
2600 entries read...
2800 entries read...
3000 entries read...
3200 entries read...
3400 entries read...
3600 entries read...
3800 entries read...
4000 entries read...
4200 entries read...
4400 entries read...
4600 entries read...
4800 entries read...
5000 entries read...
5200 entries read...
5400 entries read...
5600 entries read...
5800 entries read...
6000 entries read...
6200 entries read...
6400 entries read...
6600 entries read...
6800 entries read...
7000 entries read...
7200 entries read...
7400 entries read...
7600 entries read...
7800 entries read...
8000 entries read...
8200 entries read...
8400 entries read...
8600 entries read...
8800 entries read...
9000 entries read...
9200 entries read...
9400 entries read...
9600 entries read...
9800 entries rea

In [ ]:
from sklearn.model_selection import train_test_split
x_train, _, y_train_1, _, = train_test_split(x_train, y_train_1, train_size=0.4)

In [ ]:
#@title
## NO MULTITASKINGGGG!!!!!!!!
## NO MULTITASKINGGGG!!!!!!!!
## NO MULTITASKINGGGG!!!!!!!!
## NO MULTITASKINGGGG!!!!!!!!
## NO MULTITASKINGGGG!!!!!!!!## NO MULTITASKINGGGG!!!!!!!!
## NO MULTITASKINGGGG!!!!!!!!
## NO MULTITASKINGGGG!!!!!!!!## NO MULTITASKINGGGG!!!!!!!!




""" Functions for creating, compiling and training model"""

# Architecture:
# Main branch
# Branch1 => Decides wheter life threatening (VF or VT style) or not (SR and others)
# Branch2 => Classify the input


# labels,Rhythm
# AFb,Atrial Fibrillation
# AFt,Atrial Flutter
# SR,Sinus Rhythm
# SVT,Supraventricular Tachycardia
# VFb,Ventricular Fibrillation
# VFt,Ventricular Flutter
# VPD,Ventricular Premature Depolarizations
# VT,Ventricular Tachycardia


def gen_model():
    inputs = keras.layers.Input(shape=(1250, 1), name='input')

    main_branch = keras.layers.Conv1D(filters=3, kernel_size=6, strides=2, activation="relu") (inputs)
    main_branch = keras.layers.BatchNormalization() (main_branch)
    main_branch = keras.layers.Conv1D(filters=5, kernel_size=5, strides=2,  activation="relu") (main_branch)
    main_branch = keras.layers.BatchNormalization() (main_branch)
    main_branch = keras.layers.Conv1D(filters=10, kernel_size=4, strides=2,  activation="relu") (main_branch)
    main_branch = keras.layers.BatchNormalization() (main_branch)
    main_branch = keras.layers.Conv1D(filters=20, kernel_size=4, strides=2,  activation="relu") (main_branch)
    main_branch = keras.layers.BatchNormalization() (main_branch)
    main_branch = keras.layers.Conv1D(filters=20, kernel_size=4, strides=2,  activation="relu") (main_branch)
    main_branch = keras.layers.BatchNormalization() (main_branch)
    #main_branch = keras.layers.Conv1D(filters=32, kernel_size=1, strides=32, activation="relu") (main_branch)
    #main_branch = keras.layers.BatchNormalization() (main_branch)
    main_branch = keras.layers.Flatten() (main_branch)
    main_branch = keras.layers.Dropout(0.5) (main_branch)
    #main_branch = keras.layers.Dense(512, activation="relu") (main_branch)
    #main_branch = keras.layers.Dropout(0.1) (main_branch)
    #main_branch = keras.layers.Dense(512, activation="relu") (main_branch)
    #main_branch = keras.layers.Dropout(0.1) (main_branch)
    main_branch = keras.layers.Dense(512, activation="relu") (main_branch)
    main_branch = keras.layers.BatchNormalization() (main_branch)

    #branch1 = keras.layers.Dense(128, activation="relu") (branch1)
    #branch1 = keras.layers.BatchNormalization() (branch1)
    #branch1 = keras.layers.Dense(128, activation="relu") (branch1)
    #branch1 = keras.layers.BatchNormalization() (branch1)
    #branch1 = keras.layers.Dropout(0.2) (branch1)

    # Decides wheter life threatening (VF or VT style) or not (SR and others)
    branch1 = keras.layers.Dense(2, activation="softmax", name='task_1_output') (main_branch)

    #branch2 = keras.layers.Dense(128, activation="relu") (main_branch)
    #branch2 = keras.layers.BatchNormalization() (branch2)
    #branch2 = keras.layers.Dense(128, activation="relu") (branch2)
    #branch2 = keras.layers.BatchNormalization() (branch2)
    #branch2 = keras.layers.Dense(128, activation="relu") (branch2)
    #branch2 = keras.layers.BatchNormalization() (branch2)
    #branch2 = keras.layers.Dropout(0.2) (branch2)

    # Classify the input
    #branch2 = keras.layers.Dense(8, activation="softmax", name='task_2_output') (main_branch)
#
    ## mean
    #branch3 = keras.layers.Dense(1, activation="relu", name='task_3_output') (main_branch)
#
    ## std
    #branch4 = keras.layers.Dense(1, activation="relu", name='task_4_output') (main_branch)
#
    ## var
    #branch5 = keras.layers.Dense(1, activation="relu", name='task_5_output') (main_branch)

    model = tf.keras.Model(inputs = inputs, outputs = branch1)#, branch2, branch3, branch4, branch5])

    return model

# loss_weight -> [0, 1]
# the weight for the second task is calculated by '1 - loss_weight'
def compile_model(model, loss_weight1, loss_weight2, loss_weight3, loss_weight4, loss_weight5):
    model.compile(optimizer=tf.keras.optimizers.Adam(0.0002),
                  loss={'task_1_output': 'binary_crossentropy',
                        #'task_2_output': 'categorical_crossentropy',
                        #'task_3_output': 'mse',
                        #'task_4_output': 'mse',
                        #'task_5_output': 'mse'
                        },
                  loss_weights={'task_1_output': loss_weight1,
                                #'task_2_output': loss_weight2,
                                #'task_3_output': loss_weight3,
                                #'task_4_output': loss_weight4,
                                #'task_5_output': loss_weight5
                                },
                  metrics={'task_1_output' : ["accuracy", tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
                           #'task_2_output' : ["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
                           #'task_3_output' : "mse",
                           #'task_4_output' : "mse",
                           #'task_5_output' : "mse"
                           })
    return model

def fit_batch(gamma_values, epochs=30, batch_size=32, save_models=False, models_dir="./trained_models", verbose=0):

    history = list()
    trained_models = list()
    test_scores = list()

    start_epoch = 10
    swa = SWA(start_epoch=start_epoch,
          lr_schedule='cyclic',
          swa_lr=0.0001,
          swa_lr2=0.0005,
          swa_freq=5,
          batch_size=batch_size,
          verbose=1)

    if save_models and not os.path.isdir(models_dir):
      os.mkdir(models_dir)

    print('Starting training on batch of models for gamma values ', gamma_values, '\n\n')

    for i, gamma in enumerate(gamma_values):

        print('Training model for gamma equal to ', gamma)
        model = gen_model()
        model = compile_model(model, gamma[0], gamma[1], gamma[2], gamma[3], gamma[4])
        start = time.time()
        model_history = model.fit({'input': x_train},
                            {'task_1_output': y_train_1, #'task_2_output': y_train_2, 'task_3_output': y_train_3, 'task_4_output': y_train_4, 'task_5_output': y_train_5
                             },
                            epochs=epochs, batch_size=batch_size, verbose=verbose) # callbacks=[swa],
        print(f'Training time: {time.time() - start}\n')
        history.append(model_history)
        trained_models.append(model)
        if save_models:
          model.save(os.path.join(models_dir, f"{i}th-model"))

        test_score = model.evaluate({'input': x_test}, {'task_1_output': y_test_1,
                                                        #'task_2_output': y_test_2, 'task_3_output': y_test_3, 'task_4_output': y_test_4, 'task_5_output': y_test_5
                                                        })
        for metric, result in zip(model.metrics_names, test_score):
          print(f"{metric}: {result}")

        test_scores.append(test_score)

    return history, trained_models, test_scores

def plot_multitask_accuracies(gammas, training_history):

    counter = 0

    for history in training_history:

        print(f'\nPlotting Accuracy/Precision/Recall vs Epochs for value of gamma number {gammas[counter]}\n')
        plt.plot(range(len(history.history['task_1_output_accuracy'])), history.history['task_1_output_accuracy'], c='r', label='Task 1')
        plt.plot(range(len(history.history['task_2_output_accuracy'])), history.history['task_2_output_accuracy'], c='b', label='Task 2')
        plt.plot(range(len(history.history['task_1_output_precision'])), history.history['task_1_output_precision'], c='r', label='Task 1')
        plt.plot(range(len(history.history['task_2_output_precision'])), history.history['task_2_output_precision'], c='b', label='Task 2')
        plt.plot(range(len(history.history['task_1_output_recall'])), history.history['task_1_output_recall'], c='r', label='Task 1')
        plt.plot(range(len(history.history['task_2_output_recall'])), history.history['task_2_output_recall'], c='b', label='Task 2')
        plt.xlabel('Epochs')
        plt.legend()
        plt.show()
        counter += 1

"""Actually training the model"""
#gamma_values = [ [0.2, 0.2, 0.2, 0.2, 0.2],
#                [0.8, 0.1, 0.0333, 0.0333, 0.0333],
#                [0.5, 0.125, 0.125, 0.125, 0.125],
#                [0.5, 0.2, 0.1, 0.1, 0.1],
#                [0.6, 0.3, 0.033, 0.033, 0.033]]

gamma_values = [[1, 0, 0, 0, 0]]

#for _ in range(20):

history, trained_models, _ = fit_batch(gamma_values, epochs=30, save_models=True, models_dir="./trained_models_sixth_augmentation", verbose=1)



Starting training on batch of models for gamma values  [[1, 0, 0, 0, 0]] 


Training model for gamma equal to  [1, 0, 0, 0, 0]
Epoch 1/30
769/769 [==============================] - 14s 12ms/step - loss: 0.5012 - accuracy: 0.7758 - auc_1: 0.8523 - precision_1: 0.7758 - recall_1: 0.7758
Epoch 2/30
769/769 [==============================] - 8s 10ms/step - loss: 0.3351 - accuracy: 0.8619 - auc_1: 0.9327 - precision_1: 0.8619 - recall_1: 0.8619
Epoch 3/30
769/769 [==============================] - 9s 11ms/step - loss: 0.2645 - accuracy: 0.8918 - auc_1: 0.9567 - precision_1: 0.8918 - recall_1: 0.8918
Epoch 4/30
769/769 [==============================] - 8s 10ms/step - loss: 0.2283 - accuracy: 0.9105 - auc_1: 0.9665 - precision_1: 0.9105 - recall_1: 0.9105
Epoch 5/30
769/769 [==============================] - 8s 11ms/step - loss: 0.1944 - accuracy: 0.9238 - auc_1: 0.9748 - precision_1: 0.9238 - recall_1: 0.9238
Epoch 6/30
769/769 [==============================] - 8s 11ms/step - loss: 0.1793 

In [ ]:
len(y_train_2)

24588

In [ ]:
len(y_train_1)

9835

In [ ]:
len(x_test)

5625

NOT T TEST DOWN HERE!!!!!


In [ ]:
!pip install heartpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.5 MB/s eta 0:00:00


In [ ]:
# Normalizing data
# -*- coding: utf-8 -*-
"""arritmia_cardiaca.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1kIePtG_yixpzcbjVl5mY7nwQkAytTe_V
"""

# Credits to https://towardsdatascience.com/multi-task-learning-for-computer-vision-classification-with-keras-36c52e6243d2

# Imports
print("start imports")
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import time
import os
import csv
import random
import heartpy as hp
from sklearn import preprocessing
from tensorflow import keras
print("end imports")


# Custom stuff from https://github.com/GATECH-EIC/TinyML-Contest-Solution/blob/master/Training/select_model.py
#from helpers.cosine_annealing import CosineAnnealingScheduler
from swa.tfkeras import SWA

def norm(x):
  return (x-np.min(x))/(np.max(x)-np.min(x))

""" Functions for loading data """
def txt_to_numpy(filename, row=1250):
    file = open(filename)
    lines = file.readlines()
    datamat = np.arange(row, dtype=np.float)
    row_count = 0
    for line in lines:
        line = line.strip().split(' ')
        datamat[row_count] = line[0]
        row_count += 1

    return datamat

"""
AFb,Atrial Fibrillation
AFt,Atrial Flutter
SR,Sinus Rhythm
SVT,Supraventricular Tachycardia
VFb,Ventricular Fibrillation
VFt,Ventricular Flutter
VPD,Ventricular Premature Depolarizations
VT,Ventricular Tachycardia
"""
def txt_to_disease_type(txt):
    txt = txt.upper()
    if txt == "AFB":
      return [1, 0, 0, 0, 0, 0, 0, 0]
    if txt == "AFT":
      return [0, 1, 0, 0, 0, 0, 0, 0]
    if txt == "SR":
      return [0, 0, 1, 0, 0, 0, 0, 0]
    if txt == "SVT":
      return [0, 0, 0, 1, 0, 0, 0, 0]
    if txt == "VFB":
      return [0, 0, 0, 0, 1, 0, 0, 0]
    if txt == "VFT":
      return [0, 0, 0, 0, 0, 1, 0, 0]
    if txt == "VPD":
      return [0, 0, 0, 0, 0, 0, 1, 0]
    if txt == "VT":
      return [0, 0, 0, 0, 0, 0, 0, 1]
    raise "A string was not recognized as a valid input on txt_to_disease_type func!"

"""
def read_data(csv_path, imgs_folder="./tinyml_contest_data_training"):
    x, y1, y2 = [], [], []
    with open(csv_path, "r") as csv_file:
      reader = csv.reader(csv_file)
      next(reader)
      for item in reader: # item[0] = label; item[1] = filename
        x.append(txt_to_numpy(os.path.join(imgs_folder, item[1]), 1250))
        y1.append([1, 0] if int(item[0]) == 0 else [0, 1]) # Label for life threat
        y2.append(txt_to_disease_type(item[1].split("-")[1]))
    x, y1, y2 = sklearn.utils.shuffle(x, y1, y2)
    return np.array(x), np.array(y1), np.array(y2)
"""
def read_data(csv_path, imgs_folder="./tinyml_contest_data_training", augmentation=False, flip_peak=True, flip_time=True, add_noise=True): #, flip_peak=False, flip_time=False, add_noise=False):
    x, y1, y2, y3, y4, y5 = [], [], [], [], [], []
    print("passed here")
    with open(csv_path, "r") as csv_file:
      reader = csv.reader(csv_file)
      next(reader)
      print("Reading data...")
      counter = 0
      for item in reader: # item[0] = label; item[1] = filename
        x.append(norm(np.array(txt_to_numpy(os.path.join(imgs_folder, item[1]), 1250))))
        y1.append([1, 0] if int(item[0]) == 0 else [0, 1]) # Label for life threat
        y2.append(txt_to_disease_type(item[1].split("-")[1]))
        counter +=1
        if counter % 200 == 0:
          print(f"{counter} entries read...")
    #x, y1, y2 = sklearn.utils.shuffle(x, y1, y2)
    x, y1 = sklearn.utils.shuffle(x, y1)
    print("passei")
    x = np.array(x)
    y3 = []


    for sample in x:
      _, w = hp.process(sample, 250, bpmmin=0, bpmmax=300)
      y3.append(w['breathingrate'])



    ## y3 creation
    #for sample in x:
    #   y3.append(np.mean(sample))
    #   stdeviation = np.std(sample)
    #   y4.append(stdeviation)
    #   y5.append(stdeviation ** 2)

    if augmentation:
      #x = x + np.random.normal(0, random.random()*np.amax(x)*0.05, (len(x), 1))

      for i in range(len(x)):
        flip_p = random.random()
        flip_t = random.random()
        if flip_p < 0.5 and flip_peak:
          x[i] = -x[i]
        if flip_t < 0.5 and flip_time:
          x[i] = np.flip(x[i])
        if add_noise:
          print(f"i={i}")
          max_peak = x[i].max() * 0.05
          factor = random.random()
        # factor = 1
          noise = np.random.normal(0, factor * max_peak, x[0].shape)
          x[i] = x[i] + noise
    return x, np.array(y1), np.array(y2), np.array(y3), np.array(y4), np.array(y5)


""" Loading data """
x_train, y_train_1, y_train_2, y_train_3, y_train_4, y_train_5 = read_data("./data_indices/train_indice.csv", augmentation=False) #augmentation=True, flip_peak=True, flip_time=True, add_noise=True)
x_test, y_test_1, y_test_2, y_test_3, y_test_4, y_test_5 = read_data("./data_indices/test_indice.csv")
print("Finished reading data!")


start imports
end imports
passed here
Reading data...
200 entries read...


<ipython-input-28-9e22cc1e749d>:40: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  datamat = np.arange(row, dtype=np.float)


400 entries read...
600 entries read...
800 entries read...
1000 entries read...
1200 entries read...
1400 entries read...
1600 entries read...
1800 entries read...
2000 entries read...
2200 entries read...
2400 entries read...
2600 entries read...
2800 entries read...
3000 entries read...
3200 entries read...
3400 entries read...
3600 entries read...
3800 entries read...
4000 entries read...
4200 entries read...
4400 entries read...
4600 entries read...
4800 entries read...
5000 entries read...
5200 entries read...
5400 entries read...
5600 entries read...
5800 entries read...
6000 entries read...
6200 entries read...
6400 entries read...
6600 entries read...
6800 entries read...
7000 entries read...
7200 entries read...
7400 entries read...
7600 entries read...
7800 entries read...
8000 entries read...
8200 entries read...
8400 entries read...
8600 entries read...
8800 entries read...
9000 entries read...
9200 entries read...
9400 entries read...
9600 entries read...
9800 entries rea

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:5246: RuntimeWarning: Mean of empty slice.
  result = super().mean(axis=axis, dtype=dtype, **kwargs)[()]
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3757: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/scipy/interpolate/_fitpack2.py:298: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:264: RuntimeWarning: 

passed here
Reading data...
200 entries read...
400 entries read...
600 entries read...
800 entries read...
1000 entries read...
1200 entries read...
1400 entries read...
1600 entries read...
1800 entries read...
2000 entries read...
2200 entries read...
2400 entries read...
2600 entries read...
2800 entries read...
3000 entries read...
3200 entries read...
3400 entries read...
3600 entries read...
3800 entries read...
4000 entries read...
4200 entries read...
4400 entries read...
4600 entries read...
4800 entries read...
5000 entries read...
5200 entries read...
5400 entries read...
5600 entries read...
passei
Finished reading data!


In [ ]:
print(y_train_3)
where_are_NaNs = np.isnan(y_train_3)
y_train_3[where_are_NaNs] = 0

[0.4456328  0.         0.31289111 ... 3.08641975 0.26315789 0.30978934]


In [ ]:
# Trying to improve multitask!!!

#best result --> no augmentation

""" Functions for creating, compiling and training model"""

# Architecture:
# Main branch
# Branch1 => Decides wheter life threatening (VF or VT style) or not (SR and others)
# Branch2 => Classify the input


# labels,Rhythm
# AFb,Atrial Fibrillation
# AFt,Atrial Flutter
# SR,Sinus Rhythm
# SVT,Supraventricular Tachycardia
# VFb,Ventricular Fibrillation
# VFt,Ventricular Flutter
# VPD,Ventricular Premature Depolarizations
# VT,Ventricular Tachycardia


def gen_model():
    inputs = keras.layers.Input(shape=(1250, 1), name='input')

    main_branch = keras.layers.Conv1D(filters=3, kernel_size=6, strides=2, activation="relu") (inputs)
    main_branch = keras.layers.BatchNormalization() (main_branch)
    main_branch = keras.layers.Conv1D(filters=5, kernel_size=5, strides=2,  activation="relu") (main_branch)
    main_branch = keras.layers.BatchNormalization() (main_branch)
    main_branch = keras.layers.Conv1D(filters=10, kernel_size=4, strides=2,  activation="relu") (main_branch)
    main_branch = keras.layers.BatchNormalization() (main_branch)
    main_branch = keras.layers.Conv1D(filters=20, kernel_size=4, strides=2,  activation="relu") (main_branch)
    main_branch = keras.layers.BatchNormalization() (main_branch)
    main_branch = keras.layers.Conv1D(filters=20, kernel_size=4, strides=2,  activation="relu") (main_branch)
    main_branch = keras.layers.BatchNormalization() (main_branch)
    #main_branch = keras.layers.Conv1D(filters=32, kernel_size=1, strides=32, activation="relu") (main_branch)
    #main_branch = keras.layers.BatchNormalization() (main_branch)
    main_branch = keras.layers.Flatten() (main_branch)
    main_branch = keras.layers.Dropout(0.5) (main_branch)
    #main_branch = keras.layers.Dense(512, activation="relu") (main_branch)
    #main_branch = keras.layers.Dropout(0.1) (main_branch)
    #main_branch = keras.layers.Dense(512, activation="relu") (main_branch)
    #main_branch = keras.layers.Dropout(0.1) (main_branch)
    main_branch = keras.layers.Dense(512, activation="relu") (main_branch)
    main_branch = keras.layers.BatchNormalization() (main_branch)

    #branch1 = keras.layers.Dense(128, activation="relu") (branch1)
    #branch1 = keras.layers.BatchNormalization() (branch1)
    #branch1 = keras.layers.Dense(128, activation="relu") (branch1)
    #branch1 = keras.layers.BatchNormalization() (branch1)
    #branch1 = keras.layers.Dropout(0.2) (branch1)

    # Decides wheter life threatening (VF or VT style) or not (SR and others)
    branch1 = keras.layers.Dense(2, activation="softmax", name='task_1_output') (main_branch)

    #branch2 = keras.layers.Dense(128, activation="relu") (main_branch)
    #branch2 = keras.layers.BatchNormalization() (branch2)
    #branch2 = keras.layers.Dense(128, activation="relu") (branch2)
    #branch2 = keras.layers.BatchNormalization() (branch2)
    #branch2 = keras.layers.Dense(128, activation="relu") (branch2)
    #branch2 = keras.layers.BatchNormalization() (branch2)
    #branch2 = keras.layers.Dropout(0.2) (branch2)

    # Classify the input
    branch2 = keras.layers.Dense(8, activation="softmax", name='task_2_output') (main_branch)

    # mean
    branch3 = keras.layers.Dense(1, activation="relu", name='task_3_output') (main_branch)

    ## std
    #branch4 = keras.layers.Dense(1, activation="relu", name='task_4_output') (main_branch)
#
    ## var
    #branch5 = keras.layers.Dense(1, activation="relu", name='task_5_output') (main_branch)

    model = tf.keras.Model(inputs = inputs, outputs = [branch1, branch2, branch3,]) #branch4, branch5])

    return model

# loss_weight -> [0, 1]
# the weight for the second task is calculated by '1 - loss_weight'
def compile_model(model, loss_weight1, loss_weight2, loss_weight3, loss_weight4, loss_weight5):
    model.compile(optimizer=tf.keras.optimizers.Adam(0.0002),
                  loss={'task_1_output': 'binary_crossentropy',
                        'task_2_output': 'categorical_crossentropy',
                        'task_3_output': 'mse',
                        #'task_4_output': 'mse',
                        #'task_5_output': 'mse'
                        },
                  loss_weights={'task_1_output': loss_weight1,
                                'task_2_output': loss_weight2,
                                'task_3_output': loss_weight3,
                                #'task_4_output': loss_weight4,
                                #'task_5_output': loss_weight5
                                },
                  metrics={'task_1_output' : ["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
                           'task_2_output' : ["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
                           'task_3_output' : "mse",
                           #'task_4_output' : "mse",
                           #'task_5_output' : "mse"
                           })
    return model

def fit_batch(gamma_values, epochs=30, batch_size=32, save_models=False, models_dir="./trained_models", verbose=0):

    history = list()
    trained_models = list()
    test_scores = list()

    start_epoch = 10
    swa = SWA(start_epoch=start_epoch,
          lr_schedule='cyclic',
          swa_lr=0.0001,
          swa_lr2=0.0005,
          swa_freq=5,
          batch_size=batch_size,
          verbose=1)

    if save_models and not os.path.isdir(models_dir):
      os.mkdir(models_dir)

    print('Starting training on batch of models for gamma values ', gamma_values, '\n\n')

    for i, gamma in enumerate(gamma_values):

        print('Training model for gamma equal to ', gamma)
        model = gen_model()
        model = compile_model(model, gamma[0], gamma[1], gamma[2], gamma[3], gamma[4])
        start = time.time()
        model_history = model.fit({'input': x_train},
                            {'task_1_output': y_train_1, 'task_2_output': y_train_2, 'task_3_output': y_train_3,
                             #'task_4_output': y_train_4, 'task_5_output': y_train_5
                             },
                            epochs=epochs, batch_size=batch_size, verbose=verbose) # callbacks=[swa],
        print(f'Training time: {time.time() - start}\n')
        history.append(model_history)
        trained_models.append(model)
        if save_models:
          model.save(os.path.join(models_dir, f"{i}th-model"))

        test_score = model.evaluate({'input': x_test}, {'task_1_output': y_test_1, 'task_2_output': y_test_2, 'task_3_output': y_test_3,
                                                        #'task_4_output': y_test_4, 'task_5_output': y_test_5
                                                        })
        for metric, result in zip(model.metrics_names, test_score):
          print(f"{metric}: {result}")

        test_scores.append(test_score)

    return history, trained_models, test_scores

def plot_multitask_accuracies(gammas, training_history):

    counter = 0

    for history in training_history:

        print(f'\nPlotting Accuracy/Precision/Recall vs Epochs for value of gamma number {gammas[counter]}\n')
        plt.plot(range(len(history.history['task_1_output_accuracy'])), history.history['task_1_output_accuracy'], c='r', label='Task 1')
        plt.plot(range(len(history.history['task_2_output_accuracy'])), history.history['task_2_output_accuracy'], c='b', label='Task 2')
        plt.plot(range(len(history.history['task_1_output_precision'])), history.history['task_1_output_precision'], c='r', label='Task 1')
        plt.plot(range(len(history.history['task_2_output_precision'])), history.history['task_2_output_precision'], c='b', label='Task 2')
        plt.plot(range(len(history.history['task_1_output_recall'])), history.history['task_1_output_recall'], c='r', label='Task 1')
        plt.plot(range(len(history.history['task_2_output_recall'])), history.history['task_2_output_recall'], c='b', label='Task 2')
        plt.xlabel('Epochs')
        plt.legend()
        plt.show()
        counter += 1

"""Actually training the model"""
#gamma_values = [ [0.2, 0.2, 0.2, 0.2, 0.2],
#                [0.8, 0.1, 0.0333, 0.0333, 0.0333],
#                [0.5, 0.125, 0.125, 0.125, 0.125],
#                [0.5, 0.2, 0.1, 0.1, 0.1],
#                [0.6, 0.3, 0.033, 0.033, 0.033]]

gamma_values = [ [0.7, 0.2, 0.1, 0, 0],]

for _ in range(20):
  history, trained_models, _ = fit_batch(gamma_values, epochs=20, save_models=True, models_dir="./trained_models_sixth_augmentation", verbose=0)


Starting training on batch of models for gamma values  [[0.7, 0.2, 0.1, 0, 0]] 


Training model for gamma equal to  [0.7, 0.2, 0.1, 0, 0]
Training time: 205.81774020195007



176/176 [==============================] - 2s 5ms/step - loss: nan - task_1_output_loss: 0.1691 - task_2_output_loss: 1.3755 - task_3_output_loss: nan - task_1_output_accuracy: 0.9586 - task_1_output_precision_12: 0.9586 - task_1_output_recall_12: 0.9586 - task_2_output_accuracy: 0.4146 - task_2_output_precision_13: 0.4184 - task_2_output_recall_13: 0.1108 - task_3_output_mse: nan
loss: nan
task_1_output_loss: 0.16906456649303436
task_2_output_loss: 1.3755264282226562
task_3_output_loss: nan
task_1_output_accuracy: 0.9585777521133423
task_1_output_precision_12: 0.9585777521133423
task_1_output_recall_12: 0.9585777521133423
task_2_output_accuracy: 0.41457778215408325
task_2_output_precision_13: 0.4184015989303589
task_2_output_recall_13: 0.110755555331707
task_3_output_mse: nan
Starting training on batch of models for gamma values  [[0.7, 0.2, 0.1, 0, 0]] 


Training model for gamma equal to  [0.7, 0.2, 0.1, 0, 0]
Training time: 206.77389812469482



176/176 [==============================] - 1s 5ms/step - loss: nan - task_1_output_loss: 0.1740 - task_2_output_loss: 1.3772 - task_3_output_loss: nan - task_1_output_accuracy: 0.9547 - task_1_output_precision_14: 0.9547 - task_1_output_recall_14: 0.9547 - task_2_output_accuracy: 0.4133 - task_2_output_precision_15: 0.4001 - task_2_output_recall_15: 0.1214 - task_3_output_mse: nan
loss: nan
task_1_output_loss: 0.1739819496870041
task_2_output_loss: 1.3772070407867432
task_3_output_loss: nan
task_1_output_accuracy: 0.9546666741371155
task_1_output_precision_14: 0.9546666741371155
task_1_output_recall_14: 0.9546666741371155
task_2_output_accuracy: 0.41333332657814026
task_2_output_precision_15: 0.4001171588897705
task_2_output_recall_15: 0.12142222374677658
task_3_output_mse: nan
Starting training on batch of models for gamma values  [[0.7, 0.2, 0.1, 0, 0]] 


Training model for gamma equal to  [0.7, 0.2, 0.1, 0, 0]
Training time: 179.913476228714



176/176 [==============================] - 2s 6ms/step - loss: nan - task_1_output_loss: 0.2069 - task_2_output_loss: 1.3719 - task_3_output_loss: nan - task_1_output_accuracy: 0.9545 - task_1_output_precision_16: 0.9545 - task_1_output_recall_16: 0.9545 - task_2_output_accuracy: 0.4252 - task_2_output_precision_17: 0.4327 - task_2_output_recall_17: 0.1298 - task_3_output_mse: nan
loss: nan
task_1_output_loss: 0.2069026231765747
task_2_output_loss: 1.3719254732131958
task_3_output_loss: nan
task_1_output_accuracy: 0.9544888734817505
task_1_output_precision_16: 0.9544888734817505
task_1_output_recall_16: 0.9544888734817505
task_2_output_accuracy: 0.42524445056915283
task_2_output_precision_17: 0.432720810174942
task_2_output_recall_17: 0.12977777421474457
task_3_output_mse: nan
Starting training on batch of models for gamma values  [[0.7, 0.2, 0.1, 0, 0]] 


Training model for gamma equal to  [0.7, 0.2, 0.1, 0, 0]
Training time: 179.65370178222656



176/176 [==============================] - 2s 6ms/step - loss: nan - task_1_output_loss: 0.2024 - task_2_output_loss: 1.3831 - task_3_output_loss: nan - task_1_output_accuracy: 0.9547 - task_1_output_precision_18: 0.9547 - task_1_output_recall_18: 0.9547 - task_2_output_accuracy: 0.4098 - task_2_output_precision_19: 0.3990 - task_2_output_recall_19: 0.1120 - task_3_output_mse: nan
loss: nan
task_1_output_loss: 0.20244917273521423
task_2_output_loss: 1.3831435441970825
task_3_output_loss: nan
task_1_output_accuracy: 0.9546666741371155
task_1_output_precision_18: 0.9546666741371155
task_1_output_recall_18: 0.9546666741371155
task_2_output_accuracy: 0.40977779030799866
task_2_output_precision_19: 0.39898669719696045
task_2_output_recall_19: 0.1120000034570694
task_3_output_mse: nan
Starting training on batch of models for gamma values  [[0.7, 0.2, 0.1, 0, 0]] 


Training model for gamma equal to  [0.7, 0.2, 0.1, 0, 0]
Training time: 206.30949330329895



...

176/176 [==============================] - 2s 6ms/step - loss: nan - task_1_output_loss: 0.1801 - task_2_output_loss: 1.3919 - task_3_output_loss: nan - task_1_output_accuracy: 0.9483 - task_1_output_precision_22: 0.9483 - task_1_output_recall_22: 0.9483 - task_2_output_accuracy: 0.3874 - task_2_output_precision_23: 0.3623 - task_2_output_recall_23: 0.1239 - task_3_output_mse: nan
loss: nan
task_1_output_loss: 0.180060476064682
task_2_output_loss: 1.3918795585632324
task_3_output_loss: nan
task_1_output_accuracy: 0.9482666850090027
task_1_output_precision_22: 0.9482666850090027
task_1_output_recall_22: 0.9482666850090027
task_2_output_accuracy: 0.3873777687549591
task_2_output_precision_23: 0.3622661232948303
task_2_output_recall_23: 0.12391111254692078
task_3_output_mse: nan
Starting training on batch of models for gamma values  [[0.7, 0.2, 0.1, 0, 0]] 


Training model for gamma equal to  [0.7, 0.2, 0.1, 0, 0]
Training time: 205.51400423049927



176/176 [==============================] - 2s 6ms/step - loss: nan - task_1_output_loss: 0.1856 - task_2_output_loss: 1.3833 - task_3_output_loss: nan - task_1_output_accuracy: 0.9552 - task_1_output_precision_24: 0.9552 - task_1_output_recall_24: 0.9552 - task_2_output_accuracy: 0.4119 - task_2_output_precision_25: 0.4016 - task_2_output_recall_25: 0.1371 - task_3_output_mse: nan
loss: nan
task_1_output_loss: 0.1856347620487213
task_2_output_loss: 1.3833025693893433
task_3_output_loss: nan
task_1_output_accuracy: 0.9552000164985657
task_1_output_precision_24: 0.9552000164985657
task_1_output_recall_24: 0.9552000164985657
task_2_output_accuracy: 0.41191110014915466
task_2_output_precision_25: 0.40156251192092896
task_2_output_recall_25: 0.13706666231155396
task_3_output_mse: nan
Starting training on batch of models for gamma values  [[0.7, 0.2, 0.1, 0, 0]] 


Training model for gamma equal to  [0.7, 0.2, 0.1, 0, 0]
Training time: 206.06904220581055



176/176 [==============================] - 2s 6ms/step - loss: nan - task_1_output_loss: 8.2136 - task_2_output_loss: 28.7944 - task_3_output_loss: nan - task_1_output_accuracy: 0.4247 - task_1_output_precision_26: 0.4247 - task_1_output_recall_26: 0.4247 - task_2_output_accuracy: 0.4690 - task_2_output_precision_27: 0.4690 - task_2_output_recall_27: 0.4690 - task_3_output_mse: nan
loss: nan
task_1_output_loss: 8.213595390319824
task_2_output_loss: 28.794361114501953
task_3_output_loss: nan
task_1_output_accuracy: 0.42471110820770264
task_1_output_precision_26: 0.42471110820770264
task_1_output_recall_26: 0.42471110820770264
task_2_output_accuracy: 0.46897777915000916
task_2_output_precision_27: 0.46897777915000916
task_2_output_recall_27: 0.46897777915000916
task_3_output_mse: nan
Starting training on batch of models for gamma values  [[0.7, 0.2, 0.1, 0, 0]] 


Training model for gamma equal to  [0.7, 0.2, 0.1, 0, 0]
Training time: 205.7227213382721



176/176 [==============================] - 2s 8ms/step - loss: nan - task_1_output_loss: 0.2030 - task_2_output_loss: 1.3718 - task_3_output_loss: nan - task_1_output_accuracy: 0.9556 - task_1_output_precision_28: 0.9556 - task_1_output_recall_28: 0.9556 - task_2_output_accuracy: 0.4276 - task_2_output_precision_29: 0.4334 - task_2_output_recall_29: 0.1268 - task_3_output_mse: nan
loss: nan
task_1_output_loss: 0.2029951810836792
task_2_output_loss: 1.3718466758728027
task_3_output_loss: nan
task_1_output_accuracy: 0.9555555582046509
task_1_output_precision_28: 0.9555555582046509
task_1_output_recall_28: 0.9555555582046509
task_2_output_accuracy: 0.42755556106567383
task_2_output_precision_29: 0.4334346652030945
task_2_output_recall_29: 0.12675555050373077
task_3_output_mse: nan
Starting training on batch of models for gamma values  [[0.7, 0.2, 0.1, 0, 0]] 


Training model for gamma equal to  [0.7, 0.2, 0.1, 0, 0]


KeyboardInterrupt: ignored